In [4]:
import gzip
import json
from typing import Any, Generator
from dataclasses import dataclass

from functional import seq  # type: ignore[reportMissingTypeStubs]

import boto3  # type: ignore[reportMissingTypeStubs]
from botocore import UNSIGNED  # type: ignore[reportMissingTypeStubs]
from botocore.client import Config  # type: ignore[reportMissingTypeStubs]

## Configuration

In [5]:
# S3 constants
BUCKET_NAME = "mccarter-blog"
FILE_PREFIX = "2023-02-18/small-files/"

## Helper functions

These are small helper functions.

Each function does exactly one task, following the Single Responsibility Principal.

We can then easily compose these functions into a simple data pipeline

In [6]:
def list_s3_bucket_file_keys(
    s3,  # type: ignore
    bucket_name: str,
    prefix: str,
) -> Generator[str, None, None]:
    """Returns a generator of file names (technically S3 bucket keys) from the given S3 bucket
    that have the given path prefix.

    Args:
        s3: The boto3 S3 client object
        bucket_name (str): The name of the S3 bucket to retrieve data from
        prefix (str): The "file path" prefix of the files (S3 bucket keys) to be returned.
            So all file paths returned from this function will have this prefix.

    Yields:
        Generator[str, None, None]: A sequence of matching file names (S3 bucket keys).
    """

    # List all objects in the bucket with specified "file/folder" prefix
    response: dict[str, Any] = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)  # type: ignore

    # Iterate through each object in the response
    for obj in response.get("Contents", []):
        # Get the object key (i.e., file path)
        key = obj.get("Key")

        # Skip directories - this function only returns file paths
        if key.endswith("/"):
            continue

        yield key


# Connect to S3 anonymously - the S3 bucket has permissions set to allow this
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))  # type: ignore

test_keys = list_s3_bucket_file_keys(s3, BUCKET_NAME, FILE_PREFIX)
test_results = seq(test_keys).take(3)  # type: ignore
expected = [
    "2023-02-18/small-files/black.json.gz",
    "2023-02-18/small-files/blue.json.gz",
    "2023-02-18/small-files/gray.json.gz",
]
assert test_results == expected, f"Expected {expected} but got {test_results}"

In [7]:
def download_s3_file(s3, bucket_name: str, key: str) -> bytes:  # type: ignore
    """Download the given file (S3 bucket key) from S3 and return the raw bytes

    Args:
        s3: The boto3 S3 client object
        bucket_name (str): The name of the S3 bucket to retrieve data from
        key (str): The S3 key (file name if you will) of the object to download

    Returns:
        bytes: The raw bytes of the given file from S3
    """
    obj_response = s3.get_object(Bucket=bucket_name, Key=key)  # type: ignore
    content: bytes = obj_response["Body"].read()  # type: ignore
    return content


# Do something with the content (e.g., print it)
content = download_s3_file(s3, BUCKET_NAME, "2023-02-18/small-files/blue.json.gz")
assert type(content) == bytes, f"Expected {type(bytes)} but got {type(content)}"

In [8]:
def read_binary_file(file_name: str) -> bytes:
    """A very simple function that reads the byte for the given file name

    Args:
        file_name (str): The name of the file to read

    Returns:
        bytes: The raw bytes of the given binary file
    """
    with open(file_name, "rb") as file:  # r for reading, b for binary file
        return file.read()

## Our business / domain layer

Here we define our simple domain model

In [9]:
@dataclass(frozen=True)
class ColourId:
    """A very small data class that holds a colour name and details."""

    id: str
    name: str
    base: str
    hex: str
    rgb: tuple[int, int, int]

### Some simple questions we can ask of our colour

In [10]:
def is_colour_reddish(colour: ColourId) -> bool:
    return colour.rgb[0] > colour.rgb[1] and colour.rgb[0] > colour.rgb[2]


def is_colour_muted(colour: ColourId) -> bool:
    return colour.rgb[0] < 128 and colour.rgb[1] < 128 and colour.rgb[2] < 128

## Data access layer

Our data access layer code is very simle - just a helper function to convert a JSON object
into an instance of our `ColourId` domain object

In [11]:
def create_colour_from_json(data: dict[str, Any]) -> ColourId:
    return ColourId(
        data["id"],
        data["name"],
        data["base"],
        data["hex"],
        data["rgb"],
    )

## The data pipeline examples

Our first data pipeline just reads some test files from our local drive.

In [ ]:
test_files = [
    "./small-files/purple.json.gz",
    "./small-files/gray.json.gz",
    "./small-files/green.json.gz",
]

result = (  # type: ignore
    seq(test_files)  # type: ignore
    .map(read_binary_file)
    .map(gzip.decompress)  # type: ignore
    .map(json.loads)  # type: ignore
    .flatten()  # type: ignore
    .to_list()  # type: ignore
)

# print(json.dumps(result, indent=2))

Our second data pipeline reads all of our .gz compressed JSON files from the S3 bucket.

We:
1. Get the list of all the files in our S3 bucket (prefix) folder 
2. Download the raw bytes of the file
3. Decompress the in-memory bytes to get the original .json file contents
4. Parse the in-memory JSON data into a Python list of dictionaries
5. Convert each dictionary into our Python `ColourId` domain object
6. Then we convert the entire thing into an in-memory list

In [16]:
result: list[ColourId] = (  # type: ignore
    seq(list_s3_bucket_file_keys(s3, BUCKET_NAME, FILE_PREFIX))  # type: ignore
    .map(lambda key: download_s3_file(s3, BUCKET_NAME, key))  # type: ignore
    .map(gzip.decompress)  # type: ignore
    .map(json.loads)  # type: ignore
    .flatten()  # type: ignore
    .map(create_colour_from_json)  # type: ignore
    .filter(is_colour_reddish)  # type: ignore
    .filter(is_colour_muted)  # type: ignore
    .to_list()  # type: ignore
)

print(f"Found {len(result)} colours")
print(result[0])

Found 76 colours
ColourId(id='b6907772-f286-4df1-b699-1d9dee8fdbb2', name='Asphalt', base='black', hex='#0C0404', rgb=[12, 4, 4])


## The usual imperative way

In [15]:
result: list[ColourId] = []

for s3_key in list_s3_bucket_file_keys(s3, BUCKET_NAME, FILE_PREFIX):  # type: ignore
    downloaded_file_bytes = download_s3_file(s3, BUCKET_NAME, s3_key)  # type: ignore
    decompressed_text = gzip.decompress(downloaded_file_bytes)
    json_data = json.loads(decompressed_text)

    for colour_dict in json_data:
        colour = create_colour_from_json(colour_dict)

        # Apply our filters
        if not is_colour_reddish(colour):
            continue
        if not is_colour_muted(colour):
            continue

        result.append(colour)

print(f"Found {len(result)} colours")
print(result[0])

Found 76 colours
ColourId(id='b6907772-f286-4df1-b699-1d9dee8fdbb2', name='Asphalt', base='black', hex='#0C0404', rgb=[12, 4, 4])
